In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', None)
pd.set_option('float_format', '{:f}'.format)

In [2]:
films = pd.read_excel('../01.Data_Collection/films_edit.xlsx')
stats = pd.read_csv('../01.Data_Collection/KOBIS_stats_201910.csv')

# KOBIS '공식통계-개봉일람'의 데이터로 매출액, 관객수 등 컬럼 값 업데이트
* 최종 기준년월: 2019년 10월 (11월 초에 10월 마감, 11월 이후 발권데이터와 마감 이후 전송된 보정데이터는 포함되지 않음)
* 집계 데이터의 정확성을 위해 2019년 8월 개봉작까지만 분석 대상으로 함 (상영기간 고려)
* 스크린수는 개봉 첫 주 최대 스크린수를 의미함

In [3]:
stats.head()

,순번,영화명,감독,제작사,배급사,개봉일,영화유형,영화형태,국적,전국스크린수,전국매출액,전국관객수,서울매출액,서울관객수,장르,등급,영화구분
0,1,극한직업,이병헌,"(주)어바웃필름,영화사 해그림 주식회사,씨제이이앤엠(주)",씨제이이앤엠(주),2019.1.23,개봉영화,장편,한국,"1,978","139,647,979,516","16,264,944","31,858,660,536","3,638,287",코미디,15세이상관람가,일반영화
1,2,신과함께-죄와 벌,김용화,"리얼라이즈픽쳐스(주),(주)덱스터스튜디오",롯데쇼핑㈜롯데엔터테인먼트,2017.12.20,개봉영화,장편,한국,"1,912","115,698,654,137","14,410,754","27,530,825,087","3,346,172",판타지,12세이상관람가,일반영화
2,3,베테랑,류승완,"(주)외유내강,(주)필름케이",씨제이이앤엠(주),2015.8.5,개봉영화,장편,한국,"1,064","105,024,756,250","13,395,400","26,766,369,950","3,290,366",액션,15세이상관람가,일반영화
3,4,암살,최동훈,(주)케이퍼필름,(주)쇼박스,2015.7.22,개봉영화,장편,한국,"1,519","98,463,132,781","12,705,700","25,306,651,181","3,155,960",액션,15세이상관람가,일반영화
4,5,신과함께-인과 연,김용화,"리얼라이즈픽쳐스(주),(주)덱스터스튜디오",롯데컬처웍스(주)롯데엔터테인먼트,2018.8.1,개봉영화,장편,한국,"2,235","102,476,768,809","12,253,247","22,905,551,759","2,636,542",판타지,12세이상관람가,일반영화


In [4]:
stats.shape

(2152, 17)

In [5]:
stats.isnull().sum()

순번         0
영화명        0
감독         7
제작사       25
배급사        2
개봉일        0
영화유형       0
영화형태       0
국적         0
전국스크린수     0
전국매출액      0
전국관객수      0
서울매출액      0
서울관객수      0
장르        40
등급         0
영화구분       0
dtype: int64

In [6]:
stats.dtypes

순번         int64
영화명       object
감독        object
제작사       object
배급사       object
개봉일       object
영화유형      object
영화형태      object
국적        object
전국스크린수    object
전국매출액     object
전국관객수     object
서울매출액     object
서울관객수     object
장르        object
등급        object
영화구분      object
dtype: object

In [7]:
# 숫자 데이터가 들어있는 컬럼들의 데이터타입을 float로 바꾼다.
digit_cols = ['전국스크린수', '전국매출액', '전국관객수', '서울매출액', '서울관객수']
stats[digit_cols] = stats[digit_cols].applymap(lambda x: float(x.replace(',','')))

In [8]:
stats.dtypes

순번          int64
영화명        object
감독         object
제작사        object
배급사        object
개봉일        object
영화유형       object
영화형태       object
국적         object
전국스크린수    float64
전국매출액     float64
전국관객수     float64
서울매출액     float64
서울관객수     float64
장르         object
등급         object
영화구분       object
dtype: object

In [9]:
# 숫자 데이터의 기초통계량
stats.describe()

,순번,전국스크린수,전국매출액,전국관객수,서울매출액,서울관객수
count,2152.000000,2152.000000,2152.000000,2152.000000,2152.000000,2152.000000
mean,1076.500000,122.278810,1951661586.572491,242117.658457,478357366.647305,57446.778346
std,621.373210,294.197920,9347164502.538885,1152468.467411,2268498601.839431,271174.881065
min,1.000000,1.000000,1000.000000,1.000000,0.000000,0.000000
25%,538.750000,1.000000,6000.000000,1.000000,0.000000,0.000000
50%,1076.500000,18.000000,839500.000000,159.000000,524000.000000,87.000000
75%,1614.250000,22.000000,18214387.500000,2440.750000,10967200.000000,1383.250000
max,2152.000000,2235.000000,139647979516.000000,16264944.000000,31858660536.000000,3638287.000000


전체 2152개의 영화 중에서 25% 이상이 전국관객수가 1임을 알 수 있다.  
-> 대부분이 에로영화이므로 이 데이터들은 제거한다.

In [10]:
stats2 = stats[stats['전국관객수']!=1]

In [11]:
len(stats2)

1441

In [12]:
# 1441개로 줄어든 상태에서 다시 기초통계량을 본다.
stats2.describe()

,순번,전국스크린수,전국매출액,전국관객수,서울매출액,서울관객수
count,1441.000000,1441.000000,1441.000000,1441.000000,1441.000000,1441.000000
mean,721.000000,182.118668,2914623488.066620,361579.798751,714382206.124219,85791.371964
std,416.125181,344.154078,11300422606.368742,1393107.760214,2741940350.083239,327735.745761
min,1.000000,1.000000,2000.000000,2.000000,0.000000,0.000000
25%,361.000000,18.000000,826000.000000,156.000000,522000.000000,81.000000
50%,721.000000,20.000000,3254000.000000,519.000000,1783500.000000,257.000000
75%,1081.000000,133.000000,145902200.000000,19405.000000,60483700.000000,7628.000000
max,1441.000000,2235.000000,139647979516.000000,16264944.000000,31858660536.000000,3638287.000000


전국관객수의 중간값인 519명 이상인 데이터만 남긴다.

In [13]:
stats3 = stats2[stats2['전국관객수'] >= 519]

In [14]:
# films와의 merge를 '영화명' 기준으로 하기 위해 같은 제목의 작품이 없는지를 먼저 살펴본다.
print(len(stats3))
print(stats3['영화명'].nunique())
print(len(films))
print(films['영화명'].nunique())

721
721
810
810


In [15]:
films.columns

Index(['순위', '영화명', '영화코드', '개봉일', '매출액', '매출액점유율', '누적매출액', '관객수', '누적관객수',
       '스크린수', '상영횟수', '대표국적', '국적', '제작사', '배급사', '등급', '장르', 'F-rated', '감독',
       '대표감독', '감독코드', '감독성별', '감독_대표역할', '감독필모', '작가', '대표작가', '대표작가(필명)',
       '작가코드', '작가성별', '작가_대표역할', '작가필모', '배우', '주연배우', '배역명', '배우코드', '배우성별',
       '배우_대표역할', '배우필모'],
      dtype='object')

In [16]:
films['개봉일'].dtype

dtype('O')

In [17]:
# films에서 2019년 9월 이후 개봉작은 제외한다.
films = films[pd.to_datetime(films['개봉일']) <= '2019-08-31']

In [18]:
len(films)

786

## merge

In [19]:
# 제목이 같은 작품이 없으므로 '영화명' 기준으로 outer merge
merged = films.merge(stats3, how='outer', on='영화명')

In [20]:
merged.isnull().sum()

순위           28
영화명           0
영화코드         26
개봉일_x        26
매출액          26
매출액점유율       28
누적매출액        26
관객수          26
누적관객수        26
스크린수         26
상영횟수         26
대표국적         28
국적_x         28
제작사_x        47
배급사_x        29
등급_x         28
장르_x         28
F-rated      26
감독_x         29
대표감독         29
감독코드         29
감독성별         31
감독_대표역할      29
감독필모         29
작가          125
대표작가        125
대표작가(필명)    125
작가코드        125
작가성별        156
작가_대표역할     125
작가필모        125
배우           26
주연배우         26
배역명         218
배우코드         26
배우성별         44
배우_대표역할      26
배우필모         97
순번           91
감독_y         94
제작사_y       108
배급사_y        92
개봉일_y        91
영화유형         91
영화형태         91
국적_y         91
전국스크린수       91
전국매출액        91
전국관객수        91
서울매출액        91
서울관객수        91
장르_y         91
등급_y         91
영화구분         91
dtype: int64

In [21]:
# 서로 붙지 않고 양쪽에 남은 영화들 중에서 관객수가 519명 이상인 것이 있는지 확인한다.
rest = merged[(merged['영화코드'].isnull()) | (merged['순번'].isnull())]
rest2 = rest[(rest['누적관객수']>=825) | (rest['전국관객수']>=825)]
rest2[['영화명', '영화코드', '누적매출액', '누적관객수', '스크린수', '상영횟수', '전국스크린수', '전국매출액', '전국관객수']].sort_values(['누적관객수', '전국관객수'], ascending=False)

,영화명,영화코드,누적매출액,누적관객수,스크린수,상영횟수,전국스크린수,전국매출액,전국관객수
403,드래곤빌리지,20177081.000000,15445500.000000,4413.000000,4.000000,48.000000,nan,nan,nan
244,우리에게 떡볶이를 먹을 권리가 있다,20150845.000000,17660200.000000,2344.000000,28.000000,417.000000,nan,nan,nan
245,과대망상자들,20150843.000000,17660200.000000,2344.000000,28.000000,417.000000,nan,nan,nan
246,소주와 아이스크림,20150844.000000,17660200.000000,2344.000000,28.000000,417.000000,nan,nan,nan
374,원스텝,20172863.000000,13002000.000000,1662.000000,33.000000,239.000000,nan,nan,nan
195,치킨게임,20161409.000000,13173800.000000,1465.000000,24.000000,262.000000,nan,nan,nan
199,세컨어카운트,20161410.000000,13173800.000000,1465.000000,24.000000,262.000000,nan,nan,nan
200,자각몽,20161411.000000,13173800.000000,1465.000000,24.000000,262.000000,nan,nan,nan
773,앨리스 죽이기,20172464.000000,9773500.000000,1260.000000,14.000000,140.000000,nan,nan,nan
54,혜경궁 홍씨,20159224.000000,10990500.000000,1217.000000,5.000000,61.000000,nan,nan,nan


데이터를 변경해서 다시 붙일 영화
* 403번 '드래곤빌리지'와 787번 '드래곤빌리지<화이트니스타운을 구하라>'는 같은 영화
* 244번 '우리에게 떡볶이를 먹을 권리가 있다', 245번 '과대망상자들', 246번 '소주와 아이스크림'은 옴니버스 영화 786번 '시선 사이'로 묶여 상영되었음
* 788번 '게이트-완결판'은 films에 이미 있는 영화 '게이트'가 재개봉했을 때의 제목

제외할 영화
* 374, 195, 199, 200, 773, 54, 307, 577번은 공식통계상 전체관객수가 519명을 넘지 않음
* 789번은 문예영화
* 791, 793, 794번은 에로영화
* 792번은 뮤지컬 공연 실황

In [22]:
# films의 '드래곤빌리지' 영화명 변경
films.loc[403, '영화명'] = '드래곤빌리지<화이트니스타운을 구하라>'

In [23]:
len(stats3)

721

In [24]:
stats3[stats3['영화명']=='시선 사이']

,순번,영화명,감독,제작사,배급사,개봉일,영화유형,영화형태,국적,전국스크린수,전국매출액,전국관객수,서울매출액,서울관객수,장르,등급,영화구분
565,566,시선 사이,"최익환,이광국","국가인권위원회,(주) 루스이소니도스",(주)영화사 진진,2016.6.9,개봉영화,옴니버스,한국,28.000000,14749200.000000,1961.000000,10536500.000000,1314.000000,드라마,12세이상관람가,독립/예술영화


In [25]:
# stats의 '시선 사이' 데이터를 3행으로 만든 후 각각의 영화명을 '우리에게 떡볶이를 먹을 권리가 있다', '과대망상자들', '소주와 아이스크림'으로 변경
stats3.loc[722] = stats3.loc[565].copy()
stats3.loc[723] = stats3.loc[565].copy()
stats3.loc[565, '영화명'] = '우리에게 떡볶이를 먹을 권리가 있다'
stats3.loc[722, '영화명'] = '과대망상자들'
stats3.loc[723, '영화명'] = '소주와 아이스크림'

In [26]:
stats3[stats3['영화명']=='게이트-완결판']

,순번,영화명,감독,제작사,배급사,개봉일,영화유형,영화형태,국적,전국스크린수,전국매출액,전국관객수,서울매출액,서울관객수,장르,등급,영화구분
616,617,게이트-완결판,신재호,삼삼공구 브라더스,(주)제이앤씨미디어그룹,2018.6.21,개봉영화,장편,한국,5.000000,1348000.000000,1339.000000,276000.000000,267.000000,범죄,15세이상관람가,일반영화


In [27]:
stats3[stats3['영화명']=='게이트']

,순번,영화명,감독,제작사,배급사,개봉일,영화유형,영화형태,국적,전국스크린수,전국매출액,전국관객수,서울매출액,서울관객수,장르,등급,영화구분
264,265,게이트,신재호,삼삼공구 브라더스,(주)제이앤씨미디어그룹,2018.2.28,개봉영화,장편,한국,208.000000,820756100.000000,101173.000000,187520600.000000,22814.000000,범죄,15세이상관람가,일반영화


In [28]:
# stats의 '게이트' 데이터에 '게이트-완결판'의 데이터를 더한 후 '게이트-완결판' 행 제거
stats3.loc[264, ['전국스크린수', '전국매출액', '전국관객수', '서울매출액', '서울관객수']] = stats.loc[264, ['전국스크린수', '전국매출액', '전국관객수', '서울매출액', '서울관객수']] + stats.loc[616, ['전국스크린수', '전국매출액', '전국관객수', '서울매출액', '서울관객수']]

In [29]:
stats3.loc[264]

순번                    265
영화명                   게이트
감독                    신재호
제작사             삼삼공구 브라더스
배급사          (주)제이앤씨미디어그룹
개봉일             2018.2.28
영화유형                 개봉영화
영화형태                   장편
국적                     한국
전국스크린수         213.000000
전국매출액    822104100.000000
전국관객수       102512.000000
서울매출액    187796600.000000
서울관객수        23081.000000
장르                     범죄
등급               15세이상관람가
영화구분                 일반영화
Name: 264, dtype: object

In [30]:
stats3 = stats3.drop(616, axis=0)

In [31]:
len(stats3)

722

In [32]:
len(films)

786

In [33]:
# 이제 inner join을 해 보자.
merged2 = films.merge(stats3, how='inner', on='영화명')

In [34]:
merged2.isnull().sum()

순위            1
영화명           0
영화코드          0
개봉일_x         0
매출액           0
매출액점유율        1
누적매출액         0
관객수           0
누적관객수         0
스크린수          0
상영횟수          0
대표국적          1
국적_x          1
제작사_x        17
배급사_x         2
등급_x          1
장르_x          1
F-rated       0
감독_x          3
대표감독          3
감독코드          3
감독성별          3
감독_대표역할       3
감독필모          3
작가           79
대표작가         79
대표작가(필명)     79
작가코드         79
작가성별        107
작가_대표역할      79
작가필모         79
배우            0
주연배우          0
배역명         166
배우코드          0
배우성별         17
배우_대표역할       0
배우필모         58
순번            0
감독_y          3
제작사_y        17
배급사_y         1
개봉일_y         0
영화유형          0
영화형태          0
국적_y          0
전국스크린수        0
전국매출액         0
전국관객수         0
서울매출액         0
서울관객수         0
장르_y          0
등급_y          0
영화구분          0
dtype: int64

In [35]:
# 이름이 같은 두 컬럼 쌍들은 서로 비교해본 후 하나를 제거
merged2[pd.to_datetime(merged2['개봉일_x']) != pd.to_datetime(merged2['개봉일_y'])]

,순위,영화명,영화코드,개봉일_x,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적_x,제작사_x,배급사_x,등급_x,장르_x,F-rated,감독_x,대표감독,감독코드,감독성별,감독_대표역할,감독필모,작가,대표작가,대표작가(필명),작가코드,작가성별,작가_대표역할,작가필모,배우,주연배우,배역명,배우코드,배우성별,배우_대표역할,배우필모,순번,감독_y,제작사_y,배급사_y,개봉일_y,영화유형,영화형태,국적_y,전국스크린수,전국매출액,전국관객수,서울매출액,서울관객수,장르_y,등급_y,영화구분


In [36]:
merged2 = merged2.drop('개봉일_y', axis=1)

In [37]:
merged2 = merged2.drop(['매출액', '매출액점유율', '누적매출액', '관객수', '누적관객수', '스크린수', '상영횟수'], axis=1)

In [38]:
merged2[merged2['대표국적'] != merged2['국적_y']]

,순위,영화명,영화코드,개봉일_x,대표국적,국적_x,제작사_x,배급사_x,등급_x,장르_x,F-rated,감독_x,대표감독,감독코드,감독성별,감독_대표역할,감독필모,작가,대표작가,대표작가(필명),작가코드,작가성별,작가_대표역할,작가필모,배우,주연배우,배역명,배우코드,배우성별,배우_대표역할,배우필모,순번,감독_y,제작사_y,배급사_y,영화유형,영화형태,국적_y,전국스크린수,전국매출액,전국관객수,서울매출액,서울관객수,장르_y,등급_y,영화구분
591,nan,선종 무문관,20183144,2018-12-13,NaN,NaN,NaN,NaN,NaN,NaN,0,윤용진,윤용진,20110475.000000,남자,감독,선종 무문관|할|내 마음 속에 작은 영웅들,윤용진,윤용진,윤용진,20110475.000000,남자,감독,선종 무문관|할|내 마음 속에 작은 영웅들,"우상전, 조용주, 고동업, 안홍진, 송유담, 임대혁, 최민재",우상전,김판수,0,NaN,배우,NaN,515,윤용진,NaN,(주)드림팩트엔터테인먼트,개봉영화,장편,한국,3.000000,25023800.000000,3012.000000,10186800.000000,1168.000000,드라마,전체관람가,독립/예술영화


In [39]:
merged2.loc[591, ['대표국적', '국적_x']] = '한국'

In [40]:
merged2 = merged2.drop('국적_y', axis=1)

In [41]:
merged2[merged2['제작사_x'] != merged2['제작사_y']]

,순위,영화명,영화코드,개봉일_x,대표국적,국적_x,제작사_x,배급사_x,등급_x,장르_x,F-rated,감독_x,대표감독,감독코드,감독성별,감독_대표역할,감독필모,작가,대표작가,대표작가(필명),작가코드,작가성별,작가_대표역할,작가필모,배우,주연배우,배역명,배우코드,배우성별,배우_대표역할,배우필모,순번,감독_y,제작사_y,배급사_y,영화유형,영화형태,전국스크린수,전국매출액,전국관객수,서울매출액,서울관객수,장르_y,등급_y,영화구분
268,46.000000,"무현, 두 도시 이야기",20162545,2016-10-26,한국,한국,"제작위원회 ,엠앤씨에프",(주)모멘텀엔터테인먼트,15세이상관람가,다큐멘터리,0,전인환,전인환,20153728.000000,남자,감독,"시민 노무현|무현, 두 도시 이야기 : 파이널 컷|무현, 두 도시 이야기|품행제로|...",NaN,NaN,NaN,nan,NaN,NaN,NaN,"김원명,노무현",노무현,이동우,10008060,남자,배우,"시민 노무현|노무현과 바보들|무현, 두 도시 이야기 : 파이널 컷|노무현입니다|무현...",227,전인환,"<무현, 두 도시 이야기> 제작위원회 ,엠앤씨에프",(주)모멘텀엔터테인먼트,개봉영화,장편,66.000000,1517735100.000000,193578.000000,521021200.000000,63745.000000,다큐멘터리,15세이상관람가,독립/예술영화
303,98.000000,문영,20158526,2017-01-12,한국,한국,NaN,KT&G 상상마당 영화사업팀 컴퍼니에스에스(주),15세이상관람가,드라마,3,김소연,김소연,20222448.000000,여자,감독,문영|문영 단편|너는 거지란다|2월 단편 상상극장-김소연 감독 단편선|오오단편극장 ...,김소연,김소연,김소연,20222448.000000,여자,감독,문영|문영 단편|너는 거지란다|2월 단편 상상극장-김소연 감독 단편선|오오단편극장 ...,"김태리,정현,박완규,박정식",김태리,NaN,20201026,여자,배우,승리호(가제)|리틀 포레스트|1987|아가씨|아가씨(확장판)|문영|문영 단편|락 아...,432,김소연,NaN,KT&G 상상마당 영화사업팀 컴퍼니에스에스(주),개봉영화,장편,23.000000,56619900.000000,7411.000000,45327400.000000,5730.000000,드라마,15세이상관람가,독립/예술영화
353,119.000000,델타 보이즈,20162972,2017-06-08,한국,한국,NaN,㈜인디스토리,15세이상관람가,"드라마,코미디",0,고봉수,고봉수,20210668.000000,남자,감독,우리마을|근본주의자|고봉수 감독 단편선|갈까부다|다영씨|튼튼이의 모험|델타 보이즈|...,고봉수,고봉수,고봉수,20210668.000000,남자,감독,우리마을|근본주의자|고봉수 감독 단편선|갈까부다|다영씨|튼튼이의 모험|델타 보이즈|...,"백승환,이웅빈,신민재,김충길,윤지혜,김소영,김정기,고성완",백승환,강일록,20277829,남자,배우,근본주의자|우리마을|튼튼이의 모험|델타 보이즈,496,고봉수,NaN,㈜인디스토리,개봉영화,장편,28.000000,26490600.000000,3705.000000,17195000.000000,2204.000000,드라마,15세이상관람가,독립/예술영화
365,151.000000,노후 대책 없다,20162971,2017-06-29,한국,한국,NaN,서울독립영화제,15세이상관람가,다큐멘터리,0,이동우,이동우,20235006.000000,남자,감독,노후 대책 없다,이동우,이동우,이동우,20235006.000000,남자,감독,노후 대책 없다,"정진용,이동우,이주영",정진용,-,20288035,남자,배우,노후 대책 없다,608,이동우,NaN,서울독립영화제,개봉영화,장편,18.000000,9751500.000000,1438.000000,6617000.000000,948.000000,다큐멘터리,15세이상관람가,독립/예술영화
405,194.000000,그리다,20173965,2017-10-26,한국,한국,NaN,㈜인디스토리,12세이상관람가,드라마,0,"장호준,이인의,박재영",이인의,10057084.000000,남자,감독,관계의 가나다에 있는 우리는|꿈의 공장|이장혁과 나|관계의 가나다에 있는 우리는|캐...,"장호준, 이인의, 박재영",장호준,장호준,10060656.000000,남자,감독,낮은 목소리 2|낮은 목소리 - 아시아에서 여성으로 산다는 것 |평양냉면|What'...,"서준영,한가림,황상경,박지연,고은민,정인기,장준휘",서준영,주용,10035605,남자,배우,실종2|어떻게 헤어질까|권법형사 : 차이나타운|스피드|비치하트애솔|썸남썸녀|방황하는...,663,"장호준,이인의,박재영",NaN,㈜인디스토리,개봉영화,옴니버스,15.000000,5996300.000000,861.000000,5024500.000000,695.000000,드라마,12세이상관람가,독립/예술영화
447,91.000000,피의 연대기,20175930,2018-01-18,한국,한국,NaN,KT&G 상상마당 영화사업팀 컴퍼니에스에스(주),12세이상관람가,다큐멘터리,3,김보람,김보람,20285310.000000,여자,감독,피의 연대기|저수지 게임|우포늪의 사람들,김보람,김보람,김보람,20285310.000000,여자,감독,피의 연대기|저수지 게임|우포늪의 사람들,"여경주,김보람,심이안,박현지,이슬기,오희정,이민지,이재명,김현아,정민주,김하원,김민...",여경주,NaN,20287912,여자,배우,피의 연대기,408,김보람,NaN,KT&G 상상마당 영화사업팀 컴퍼니에스에스(주),개봉영화,장편,38.000000,79380200.000000,10145.000000,61178500.000000,7318.000000,다큐멘터리,12세이상관람가,독립/예술영화
512,112.000000,튼튼이의 모험,20174647,2018-06-21,한국,한국,NaN,"CGV아트하우스,㈜인디스토리",15세이상관람가,코미디,0,고봉수,고봉수,20210668.000000,남자,감독,우리마을|근본주의자|고봉수 감독 단편선|갈까부다|다영씨|튼튼이의 모험|델타 보이즈|...,고봉수,고봉수,고봉수,20210668.000000,남자,감독,우리마을|근본주의자|고봉수 감독 단편선|갈까부다|다영씨|튼튼이의 모험|델타 보이즈|...,"김충길,백승환,신민재,고성완,권하늘,김윤정,김정기,박원진,윤지혜,이건창,이민주,이슬...",김충길,-,20277850,남자,배우,튼튼이의 모험|델타 보이즈|너는 아무렇지 않게 나왔겠지만,479,고봉수,NaN,"CGV아트하우스,㈜인디스토리",개봉영화,장편,88.000000,34307600.000000,4476.000000,18808100.000000,2330.000000,코미디,15세이상관람가,독립/예술영화
552,133.000000,나부야 나부야,20183386,2018-09-20,한국,한국,NaN,㈜인디스토리,전체관람가,다큐멘터리,0,최정우,최정우,20303850.000000,남자,감독,나부야 나부야,NaN,NaN,NaN,nan,NaN,NaN,NaN,"이종수,김순규",이종수,안병남,20309789,남자,배우,나부야 나부야,529,최정우,NaN,㈜인디스토리,개봉영화,장편,18.000000,14895400.000000,2765.000000,7792800.000000,1550.000000,다큐멘터리,전체관람가,독립/예술영화
562,131.000000,집의 시간들,20172388,2018-10-25,한국,한국,NaN,KT&G 상상마당 영화사업팀 컴퍼니에스에스(주),전체관람가,다큐멘터리,1,라야,라야,20281789.000000,여자,감독,집의 시간들,NaN,NaN,NaN,nan,NaN,NaN,NaN,"김기수,이인규",김기수,윤영,20319595,남자,배우,집의 시간들,523,라야,NaN,KT&G 상상마당 영화사업팀 컴퍼니에스에스(주),개봉영화,장편,20.000000,24646950.000000,2882.000000,21579950.000000,2435.000000,다큐멘터리,전체관람가,독립/예술영화
572,160.000000,마담 B,20162979,2018-11-15,한국,"한국,프랑스",NaN,영화배급협동조합 씨

In [42]:
merged2 = merged2.drop('제작사_x', axis=1)

In [43]:
merged2[merged2['배급사_x'] != merged2['배급사_y']]

,순위,영화명,영화코드,개봉일_x,대표국적,국적_x,배급사_x,등급_x,장르_x,F-rated,감독_x,대표감독,감독코드,감독성별,감독_대표역할,감독필모,작가,대표작가,대표작가(필명),작가코드,작가성별,작가_대표역할,작가필모,배우,주연배우,배역명,배우코드,배우성별,배우_대표역할,배우필모,순번,감독_y,제작사_y,배급사_y,영화유형,영화형태,전국스크린수,전국매출액,전국관객수,서울매출액,서울관객수,장르_y,등급_y,영화구분
130,80.000000,나쁜 나라,20156047,2015-12-03,한국,한국,"배급위원회,(주)시네마달",12세이상관람가,다큐멘터리,2,"김진열,이수정,정일건",김진열,10005927.000000,여자,감독,"나쁜 나라|진옥 언니 학교 가다|잊혀진 여전사|땅, 밥 만들기|남매와 진달래 |독립...",신범숙,신범숙,신범숙,20229759.000000,여자,시나리오(각본),나쁜 나라,"최경덕,박지민",최경덕,-,20228869,남자,배우,소공녀|나쁜 나라,358,"김진열,이수정,정일건",4.16 세월호 참사 시민기록위원회,"<나쁜 나라> 배급위원회,(주)시네마달",개봉영화,장편,19.000000,139403400.000000,20803.000000,60590000.000000,8452.000000,다큐멘터리,12세이상관람가,독립/예술영화
570,61.000000,출국,20182935,2018-11-14,한국,한국,NaN,15세이상관람가,드라마,0,노규엽,노규엽,20139889.000000,남자,조감독,출국|배꼽|미인도|우아한 세계,노규엽,노규엽,노규엽,20139889.000000,남자,조감독,출국|배꼽|미인도|우아한 세계,"이범수,연우진,박혁권,박주미,이현정,이종혁,이동훈,송원종,이형섭,정석원,조현수,차민...",이범수,NaN,10055581,남자,배우,자전차왕 엄복동|출국|작쩐|인천상륙작전 : 익스텐디드 에디션(확장판)|인천상륙작전|...,278,노규엽,주식회사 디에스이이디,NaN,개봉영화,장편,274.000000,639521850.000000,81722.000000,244385200.000000,32001.000000,드라마,15세이상관람가,독립/예술영화
591,nan,선종 무문관,20183144,2018-12-13,한국,한국,NaN,NaN,NaN,0,윤용진,윤용진,20110475.000000,남자,감독,선종 무문관|할|내 마음 속에 작은 영웅들,윤용진,윤용진,윤용진,20110475.000000,남자,감독,선종 무문관|할|내 마음 속에 작은 영웅들,"우상전, 조용주, 고동업, 안홍진, 송유담, 임대혁, 최민재",우상전,김판수,0,NaN,배우,NaN,515,윤용진,NaN,(주)드림팩트엔터테인먼트,개봉영화,장편,3.000000,25023800.000000,3012.000000,10186800.000000,1168.000000,드라마,전체관람가,독립/예술영화


In [44]:
merged2 = merged2.drop('배급사_x', axis=1)

In [45]:
merged2[merged2['등급_x'] != merged2['등급_y']]

,순위,영화명,영화코드,개봉일_x,대표국적,국적_x,등급_x,장르_x,F-rated,감독_x,대표감독,감독코드,감독성별,감독_대표역할,감독필모,작가,대표작가,대표작가(필명),작가코드,작가성별,작가_대표역할,작가필모,배우,주연배우,배역명,배우코드,배우성별,배우_대표역할,배우필모,순번,감독_y,제작사_y,배급사_y,영화유형,영화형태,전국스크린수,전국매출액,전국관객수,서울매출액,서울관객수,장르_y,등급_y,영화구분
591,nan,선종 무문관,20183144,2018-12-13,한국,한국,NaN,NaN,0,윤용진,윤용진,20110475.000000,남자,감독,선종 무문관|할|내 마음 속에 작은 영웅들,윤용진,윤용진,윤용진,20110475.000000,남자,감독,선종 무문관|할|내 마음 속에 작은 영웅들,"우상전, 조용주, 고동업, 안홍진, 송유담, 임대혁, 최민재",우상전,김판수,0,NaN,배우,NaN,515,윤용진,NaN,(주)드림팩트엔터테인먼트,개봉영화,장편,3.000000,25023800.000000,3012.000000,10186800.000000,1168.000000,드라마,전체관람가,독립/예술영화


In [46]:
merged2 = merged2.drop('등급_x', axis=1)

In [47]:
merged2[merged2['장르_x'] != merged2['장르_y']].sample(5)

,순위,영화명,영화코드,개봉일_x,대표국적,국적_x,장르_x,F-rated,감독_x,대표감독,감독코드,감독성별,감독_대표역할,감독필모,작가,대표작가,대표작가(필명),작가코드,작가성별,작가_대표역할,작가필모,배우,주연배우,배역명,배우코드,배우성별,배우_대표역할,배우필모,순번,감독_y,제작사_y,배급사_y,영화유형,영화형태,전국스크린수,전국매출액,전국관객수,서울매출액,서울관객수,장르_y,등급_y,영화구분
92,157.000000,덫: 치명적인 유혹,20113634,2015-09-17,한국,한국,"스릴러,멜로/로맨스,드라마",0,봉만대,봉만대,10031639.000000,남자,감독,중2라도 괜찮아|한강블루스|프랑스 영화처럼|덫: 치명적인 유혹|아티스트 봉만대|아티...,이창열,이창열,이창열,10057674.000000,남자,시나리오(각본),해피 투게더|덫: 치명적인 유혹|남자 태어나다,"유하준,한제인,강용규,김도연,김정곤,박영훈,김정곤",유하준,"시나리오 작가, 정민",10054122,남자,배우,덫: 치명적인 유혹|의뢰인|적과의 동침|여기보다 어딘가에|백문백답|시선 너머,610,봉만대,지오엔터테인먼트,골든타이드픽처스(주),개봉영화,장편,19.000000,8462000.000000,1413.000000,4605000.000000,648.000000,스릴러,청소년관람불가,독립/예술영화
172,248.000000,청춘학당2 기생난입야사,20162145,2016-03-03,한국,한국,"코미디,드라마",0,김대성,김대성,20223386.000000,남자,감독,여선생2 무삭제|우리도련님 무삭제|우리 도련님|여선생2|여자친구|감금의 시간 2 무...,박우리,박우리,박우리,20217898.000000,남자,시나리오(각본),여선생|엄마친구 2|청춘학당2 기생난입야사|감금의 시간|친절한 가정부,"전현수,지은서",전현수,이진상,20218390,남자,배우,NaN,714,김대성,(주)영화사히트,(주)영화사히트,개봉영화,장편,20.000000,3313000.000000,556.000000,1384000.000000,226.000000,코미디,청소년관람불가,일반영화
525,33.000000,인랑,20170942,2018-07-25,한국,한국,"SF,액션",0,김지운,김지운,10005852.000000,남자,감독,"인랑|밀정|라스트 스탠드|악마를 보았다|악마를 보았다 감독판|좋은 놈, 나쁜 놈, ...","김지운, 전철홍",김지운,김지운,10005852.000000,남자,감독,"인랑|밀정|라스트 스탠드|악마를 보았다|악마를 보았다 감독판|좋은 놈, 나쁜 놈, ...","강동원,한효주,정우성,김무열,최민호,한예리,신은수,허준호,문예원,김현준,이가경,박형...",강동원,임중경,10000558,남자,배우,반도|인랑|쓰나미 LA|골든슬럼버|1987|마스터|가려진 시간|검사외전|검은 사제들...,131,김지운,(주)루이스 픽쳐스,워너브러더스 코리아(주),개봉영화,장편,1085.000000,6994748173.000000,897548.000000,1768239473.000000,216427.000000,SF,15세이상관람가,일반영화
245,50.000000,달빛궁궐,20166541,2016-09-07,한국,한국,"애니메이션,판타지,어드벤처",3,김현주,김현주,20240026.000000,여자,감독,달빛궁궐|하얀물개,김현주,김현주,김현주,20240026.000000,여자,감독,달빛궁궐|하얀물개,"김서영,이하늬,권율,김슬기,신용우,이장원,시영준,이상범,이현,강시현,문유정,정주원,...",김서영,현주리 목소리,10004461,여자,배우,뽀로로 극장판 보물섬 대모험|극장판 공룡메카드: 타이니소어의 섬|런닝맨 : 풀룰루의...,237,김현주,"스튜디오홀호리(주),(주)콘텐츠판다",(주)넥스트엔터테인먼트월드(NEW),개봉영화,장편,425.000000,1134639700.000000,151360.000000,258859700.000000,31822.000000,애니메이션,전체관람가,일반영화
486,76.000000,스파키,20182221,2018-04-12,한국,한국,"애니메이션,어드벤처,코미디",1,"아론 우들리,애론 우들리",아론 우들리,10040004.000000,남자,감독,스파키|완전범죄: 키드냅,애론 우들리,애론 우들리,애론 우들리,20315335.000000,NaN,감독,스파키,"제시카 비엘,수잔 서랜든",제시카 비엘,NaN,10063343,여자,배우,충격과 공포|스파키|카인드 오브 머더|엑시덴탈 러브|여동생의 비밀|트루스 어바웃 엠...,329,"아론 우들리,애론 우들리",(주)레드로버,예지림엔터테인먼트,개봉영화,장편,312.000000,257040200.000000,35436.000000,41353300.000000,5315.000000,애니메이션,전체관람가,일반영화


In [48]:
merged2 = merged2.drop('장르_y', axis=1)

In [49]:
merged2[merged2['감독_x'] != merged2['감독_y']]

,순위,영화명,영화코드,개봉일_x,대표국적,국적_x,장르_x,F-rated,감독_x,대표감독,감독코드,감독성별,감독_대표역할,감독필모,작가,대표작가,대표작가(필명),작가코드,작가성별,작가_대표역할,작가필모,배우,주연배우,배역명,배우코드,배우성별,배우_대표역할,배우필모,순번,감독_y,제작사_y,배급사_y,영화유형,영화형태,전국스크린수,전국매출액,전국관객수,서울매출액,서울관객수,등급_y,영화구분
213,123.000000,과대망상자들,20150843,2016-06-09,한국,한국,드라마,0,신연식,신연식,10040499.000000,남자,감독,로마서 8:37|동주|프랑스 영화처럼|조류인간|개를 훔치는 완벽한 방법|배우는 배우...,NaN,NaN,NaN,nan,NaN,NaN,NaN,"정예녹,김동완,오광록,박주희,서영화,윤영민,정지훈,최희서,백수장",김동완,민,10003957,남자,배우,소리꾼|글로리데이|어떤이의 꿈|신화 라이브 3D 더 레전드 컨티뉴스|연가시|돌려차기...,566,"최익환,이광국","국가인권위원회,(주) 루스이소니도스",(주)영화사 진진,개봉영화,옴니버스,28.000000,14749200.000000,1961.000000,10536500.000000,1314.000000,12세이상관람가,독립/예술영화
214,123.000000,소주와 아이스크림,20150844,2016-06-09,한국,한국,드라마,1,이광국,이광국,10054828.000000,남자,감독,호랑이보다 무서운 겨울손님|꿈보다 해몽|로맨스 조|하하하|잘 알지도 못하면서|해변의...,NaN,NaN,NaN,nan,NaN,NaN,NaN,"정예녹,김동완,오광록,박주희,서영화,윤영민,정지훈,최희서,백수장",박주희,-,20112298,여자,배우,상류사회|인랑|오목소녀|걷기왕|간신|치욕일기|경성학교: 사라진 소녀들|워킹걸|거인|...,566,"최익환,이광국","국가인권위원회,(주) 루스이소니도스",(주)영화사 진진,개봉영화,옴니버스,28.000000,14749200.000000,1961.000000,10536500.000000,1314.000000,12세이상관람가,독립/예술영화
448,69.000000,젝스키스 에이틴,20189525,2018-01-18,한국,한국,다큐멘터리,0,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,"은지원,이재진,장수원,김재덕,강성훈,이재진",은지원,이충연,10054654,남자,배우,젝스키스 에이틴|링스 어드벤처|여고생 시집가기|세븐틴,306,NaN,(주)와이지엔터테인먼트,씨제이 씨지브이(CJ CGV)(주),개봉영화,장편,25.000000,364405500.000000,50419.000000,225891500.000000,29333.000000,전체관람가,일반영화
488,40.000000,"그날, 바다",20182352,2018-04-12,한국,한국,다큐멘터리,0,"김지영,김지영",김지영,20302876.000000,남자,감독,"그날, 바다","최진아, 김지영, 인정옥",최진아,최진아,20303650.000000,남자,프로듀서,"그날, 바다","정우성,이주용,김지영,장동원,박종대,김도연,김도영,김어준,김지영",정우성,한서린,10062025,남자,배우,"지푸라기라도 잡고 싶은 짐승들|정상회담|증인|인랑|그날, 바다|강철비|더 킹|아수라...",158,김지영,프로젝트 부,(주)엣나인필름,개봉영화,장편,611.000000,4445554528.000000,540558.000000,1157399300.000000,136574.000000,15세이상관람가,독립/예술영화
588,84.000000,트와이스랜드,20184529,2018-12-07,한국,한국,다큐멘터리,1,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,"나연,정연,모모,사나,지효,미나,다현,채영,쯔위",나연,NaN,10072145,여자,배우,트와이스랜드,364,NaN,(주)제이와이피엔터테인먼트,씨제이 씨지브이(CJ CGV)(주),개봉영화,장편,53.000000,153765000.000000,18252.000000,70742500.000000,8086.000000,전체관람가,일반영화
607,35.000000,러브 유어셀프 인 서울,20185124,2019-01-26,한국,한국,공연,0,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,"김남준,김석진,민윤기,정호석,박지민,김태형,전정국",김남준,NaN,20311775,남자,배우,브링 더 소울 : 더 무비|러브 유어셀프 인 서울|번 더 스테이지: 더 무비,193,NaN,"빅히트 엔터테인먼트,씨제이 씨지브이(CJ CGV)(주)",씨제이 씨지브이(CJ CGV)(주),개봉영화,장편,224.000000,3209682900.000000,342366.000000,1090468000.000000,112697.000000,전체관람가,일반영화


In [50]:
merged2.loc[488, '감독_x'] = '김지영'

In [51]:
merged2 = merged2.drop('감독_y', axis=1)

In [52]:
merged2.columns

Index(['순위', '영화명', '영화코드', '개봉일_x', '대표국적', '국적_x', '장르_x', 'F-rated', '감독_x',
       '대표감독', '감독코드', '감독성별', '감독_대표역할', '감독필모', '작가', '대표작가', '대표작가(필명)',
       '작가코드', '작가성별', '작가_대표역할', '작가필모', '배우', '주연배우', '배역명', '배우코드', '배우성별',
       '배우_대표역할', '배우필모', '순번', '제작사_y', '배급사_y', '영화유형', '영화형태', '전국스크린수',
       '전국매출액', '전국관객수', '서울매출액', '서울관객수', '등급_y', '영화구분'],
      dtype='object')

In [53]:
merged2 = merged2.drop(['순위', '영화유형', '순번'], axis=1).rename(columns = lambda x: x.replace('_x','').replace('_y','').strip())

In [55]:
len(merged2.columns)

37

In [56]:
cols = ['영화명', '영화코드', '개봉일', '대표국적', '국적', '장르', '등급', '영화형태', '영화구분', '제작사', '배급사', 
        '전국스크린수', '전국매출액', '전국관객수', '서울매출액', '서울관객수', 'F-rated', 
        '감독', '대표감독', '감독코드', '감독성별', '감독_대표역할', '감독필모', 
        '작가', '대표작가', '대표작가(필명)', '작가코드', '작가성별', '작가_대표역할', '작가필모', 
        '배우', '주연배우', '배역명', '배우코드', '배우성별', '배우_대표역할', '배우필모']

In [57]:
len(cols)

37

In [58]:
merged2 = merged2[cols]

In [59]:
merged2.sample()

,영화명,영화코드,개봉일,대표국적,국적,장르,등급,영화형태,영화구분,제작사,배급사,전국스크린수,전국매출액,전국관객수,서울매출액,서울관객수,F-rated,감독,대표감독,감독코드,감독성별,감독_대표역할,감독필모,작가,대표작가,대표작가(필명),작가코드,작가성별,작가_대표역할,작가필모,배우,주연배우,배역명,배우코드,배우성별,배우_대표역할,배우필모
425,아기와 나,20160522,2017-11-23,한국,한국,드라마,15세이상관람가,장편,독립/예술영화,한국영화아카데미,CGV아트하우스,62.000000,17949800.000000,2356.000000,10957100.000000,1345.000000,0,손태겸,손태겸,20112723.000000,남자,감독,아기와 나|우리 이웃 이야기|미생 프리퀄|여름방학|자전거 도둑|야간 비행|인디스데이...,손태겸,손태겸,손태겸,20112723.000000,남자,감독,아기와 나|우리 이웃 이야기|미생 프리퀄|여름방학|자전거 도둑|야간 비행|인디스데이...,"이이경,정연주,이승현,가심현,최초아,오희준,박명신,류경수,전신환,박순천",이이경,아내,20124322,남자,배우,히트맨|뷰티풀 보이스|더 펜션|괴물들|아기와 나|커튼콜|해적: 바다로 간 산적|일대...


In [61]:
merged2.shape

(699, 37)

In [64]:
merged2.to_csv('../01.Data_Collection/films_updated.csv', index=False)

In [65]:
# 공식통계에는 상영횟수 데이터가 없어서 못 넣었다. 어디서 구할 수 있지?